## Grouping all the data for the test without initial global search
The search is started from the middle of the search space because it is the expected value of the prior distribution which is defined as uniform over the parameter space $$E[U(\theta_{min},\theta_{max})] = \frac{\theta_{max}-\theta_{min}}{2}$$

### Imports & settings

In [1]:
import numpy as np
import torch
from matplotlib import pyplot as plt
import matplotlib as mpl
import pickle as pkl
from lib.utils import *
from copy import deepcopy as dcp
from os import walk
import shutil
from lib.Wilson_Cowan.parameters_info import parameter_names, parameters_lower_bound,parameters_range_bounds
mpl.rcParams['font.size'] = 18
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.facecolor'] = 'w'

### Loading the data from the CMA-ES search

In [ ]:
cma_p = '../Data/WC/from_mid/cmaes/'
cma_fs = ['cmaes_25chk_120rd_all_best_final25repeats']
target_indices = [i for i in range(0,20)]
num_std_tested = [1]
results = [[None for _ in range(4)] for _ in range(1)]
runtime_results = [[None for _ in range(4)] for _ in range(1)]
init_files = []
runtime_CMA = []
all_std_errors = []
for iii, cma_f in enumerate(cma_fs):
    cmaes_log = get_log(cma_p,cma_f)
    for ii in range(num_std_tested[iii]):
        cmaes_errors = []
        run_time = []
        for j in range(ii,20*num_std_tested[iii],num_std_tested[iii]):
            errors = -cmaes_log[j]['fit_history']['error']
            run_time.append(cmaes_log[j]['overall_runtime'])
            cmaes_errors.append(errors)
        cmaes_errors = torch.cat(cmaes_errors, dim = 0).mean(-1).unsqueeze(0)
        results[ii][3] = cmaes_errors
        run_time = torch.cat(run_time, dim = 0).mean(-1)
        runtime_results[ii][3] = run_time.unsqueeze(0)
print(results[0][3].shape)
print(runtime_results[0][3].shape)

### Loading the data from the PSO search

In [3]:
global_search_path_PSO = '../Data/WC/from_mid/PSO/'
PSO_paths = [
    'noinit-20targets_blw0.15_chunk25_120rds_PSO',
]
search_logs = [get_log(global_search_path_PSO, PSO_paths[i]) for i in range(len(PSO_paths))]
for i in range(len(PSO_paths)):
    results[i][2] = -search_logs[i][0]['fit_history']['error'][target_indices].mean(-1).unsqueeze(0)
    runtime_results[i][2] = search_logs[i][0]['overall_runtime'].squeeze().unsqueeze(0)
pso_fits = [search_logs[i][0]['best_fit']['x'][target_indices] for i in range(len(PSO_paths))]
#best_PSO_mean_errors = [errors[i].mean(1, keepdims = True).mean(0, keepdims = True) for i in range(len(PSO_paths))]
print(results[0][2].shape)
print(runtime_results[0][2].shape)

### Loading the data from the RS & SA searches

In [4]:
random_search_paths = [
    [
        ['noinit-20targets_blw0.14_chunk25_120rds_pure', 0],
		['noinit-20targets_blw0.11_chunk25_120rds_SA', 1],
    ]
]
global_search_path = '../Data/WC/from_mid/simple/'
res_index = [0,1]
for ii in range(len(random_search_paths)):
    settings = [None for _ in range(6)]
    for j, path in enumerate(random_search_paths[ii]):
        file_er = get_log(global_search_path, path[0])
        err = -file_er[path[1]]['fit_history']['error'][target_indices][:,:,:121,:].mean(-1).unsqueeze(0)
        settings[path[1]] = file_er[path[1]]['settings']
        results[ii][res_index[path[1]]] = err
        runtime_results[ii][res_index[path[1]]] = file_er[path[1]]['overall_runtime'].squeeze().unsqueeze(0)
print(results[0][0].shape)
print(runtime_results[0][0].shape)

### Concatenating all the data

In [ ]:
results = torch.cat(results[0], dim = 0)
print(results.shape)
runtime_results = torch.cat(runtime_results[0], dim = 0)
print(runtime_results.shape)

### Saving the file to be loaded in the HH_analyze_data.ipynb file

In [ ]:
combined_data_path = "../Data/"
results_log = {
    'results': results,
    'runtime': runtime_results
}
save_log(results_log, combined_data_path, 'WC_no_init')